# Análise dos Ritmos de contágio municipais para o G1 

Para uma melhor compreensão da situação dos municípios no combate a Pandemia da Covid-19, analisamos as 5% cidades com mais mortes para a verificação dos ritmos de contágio usando os dados do datasource do coronacidades. Usando indicadores como a quantidade de mortes e o ritmo de contágio de cada município podemos formar uma compreensão de como as mortes causadas pelo coronavírus e a variação de Rt se relaciona nos municípios onde a situação é mais grave, além da situação dos munícipios na curva de mortes.

*Code made by: Francisco Nogueira*

<img src="https://coronacidades.org/wp-content/uploads/2020/05/logo-coronacidades-azul.svg" width="150" height="75" align="left"></img>

## Importando as bibliotecas necessárias

In [1]:
%reload_ext autoreload
%autoreload 2
import sys
sys.path.append("..")

import numpy as np
import pandas as pd
pd.options.display.max_columns = 999
import warnings

warnings.filterwarnings('ignore')

# Plotting
import plotly
import plotly.graph_objs as go
import cufflinks as cf
plotly.offline.init_notebook_mode(connected=True)

# Centering and fixing title
def iplottitle(title, width=40):
    return '<br>'.join(textwrap.wrap(title, width))


In [2]:
from scripts.auxiliar import *
import matplotlib.pyplot as plt

## Load Data

Aqui estaremos os usando os dados do datasource padrão do coronacidades para fazer essa análise.

In [3]:
df = pd.read_csv('http://datasource.coronacidades.org:7000/br/cities/farolcovid/main')
df = df.sort_values('city_name')\
                    .reset_index(drop=True)

df.head()

,city_id,city_name,state_id,state_name,population,health_system_region,last_updated_number_beds,author_number_beds,number_beds,last_updated_number_ventilators,author_number_ventilators,number_ventilators,confirmed_cases,active_cases,deaths,notification_rate,state_notification_rate,last_updated_subnotification,subnotification_place_type,subnotification_rate,subnotification_rank,subnotification_classification,last_updated_rt,rt_10days_ago_low,rt_10days_ago_high,rt_10days_ago_most_likely,rt_17days_ago_low,rt_17days_ago_high,rt_17days_ago_most_likely,rt_classification,rt_10days_ago_avg,rt_17days_ago_avg,rt_ratio_week_avg,rt_growth,last_updated_inloco,inloco_today_7days_avg,inloco_last_week_7days_avg,inloco_ratio_week_avgs,inloco_growth,dday_beds_best,dday_beds_worst,dday_classification,dday_beds_best_months,overall_alert,data_last_refreshed
0,5200050,Abadia de Goiás,GO,Goiás,8773,Central,2020-04-01,DataSUS,0,2020-04-01,DataSUS,0.0,10.0,6.0,0.0,0.660313,0.660313,2020-06-10 12:53:24,state,0.339687,NaN,NaN,2020-06-10 13:35:27,0.57,2.67,1.54,0.61,4.04,2.09,NaN,1.70,NaN,NaN,NaN,2020-06-10 12:54:43,0.341017,0.360402,0.946212,estabilizando,NaN,NaN,NaN,NaN,NaN,2020-06-10 13:36:23
1,3100104,Abadia dos Dourados,MG,Minas Gerais,6989,Uberlândia,2020-04-01,DataSUS,11,2020-04-01,DataSUS,0.0,5.0,0.0,0.0,0.820360,0.820360,2020-06-10 12:53:24,state,0.179640,NaN,NaN,2020-06-10 13:35:27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-06-10 12:54:43,0.399017,0.424621,0.939701,estabilizando,NaN,NaN,NaN,NaN,NaN,2020-06-10 13:36:23
2,5200100,Abadiânia,GO,Goiás,20042,Pirineus,2020-04-01,DataSUS,9,2020-04-01,DataSUS,0.0,14.0,21.0,0.0,0.660313,0.660313,2020-06-10 12:53:24,state,0.339687,NaN,NaN,2020-06-10 13:35:27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-06-10 12:54:43,0.383641,0.383686,0.999882,estabilizando,NaN,NaN,NaN,NaN,NaN,2020-06-10 13:36:23
3,1500107,Abaetetuba,PA,Pará,157698,Alça Viária,2020-04-01,DataSUS,214,2020-04-01,DataSUS,10.0,1735.0,785.0,68.0,0.523409,0.294076,2020-06-10 12:53:24,city,0.476591,49.0,bom,2020-06-10 13:35:27,1.63,1.85,1.75,1.59,1.91,1.77,ruim,1.97,1.648571,1.194974,piorando,2020-06-10 12:54:43,0.444223,0.477182,0.930930,estabilizando,10.0,9.0,ruim,1.0,alto,2020-06-10 13:36:23
4,3100203,Abaeté,MG,Minas Gerais,23237,Sete Lagoas,2020-04-01,DataSUS,32,2020-04-01,DataSUS,2.0,2.0,0.0,0.0,0.820360,0.820360,2020-06-10 12:53:24,state,0.179640,NaN,NaN,2020-06-10 13:35:27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-06-10 12:54:43,0.344277,0.363129,0.948084,estabilizando,NaN,NaN,NaN,NaN,NaN,2020-06-10 13:36:23


In [4]:
# Indicador sobre atualização dos dados e total de mortos
print("Total de mortos: ", df['deaths'].sum())

print("Atualizado em: ", df['data_last_refreshed'].max())

print("Rt atualizado em: ", df['last_updated_subnotification'].min())

Total de mortos:  38264.0
Atualizado em:  2020-06-10 13:36:23
Rt atualizado em:  2020-06-10 12:53:24


Carregando os dados por estado

In [5]:
dfstates = pd.read_csv('http://datasource.coronacidades.org:7000/br/states/farolcovid/main')

Filtrando dados que apresentam cálculo de Rt<p>
O rt_ratio_week_avg é a coluna que representa a razão entre o Rt médio nos últimos 10 dias e o Rt médio nos ultimos 17 dias, portanto descreve a variação do Rt. Se o ratio é maior que 1, então o local teve um Rt em média maior nos últimos 10 dias do que nos últimos 17, portanto houve um crescimento médio. O inverso vale para um ratio menor que 1.

In [6]:
dfna = df.loc[df['rt_ratio_week_avg'].isna()]
dfnna = df.loc[~df['rt_ratio_week_avg'].isna()]

In [7]:
# 95° Percentil
p95_deaths = np.percentile(dfnna['deaths'],95)
print(f"95° Percentil de Mortos: {p95_deaths:.2f}")

95° Percentil de Mortos: 35.00


Filtrando os municípios que concentram mais mortes

In [8]:
dfnfilt = dfnna.query(f"deaths >= {p95_deaths}")\
            .sort_values(['rt_ratio_week_avg'])\
            .reset_index(drop=True)

print("Total de Municípios analisado: %d" % dfnfilt.shape[0])

Total de Municípios analisado: 127


In [9]:
dfnfilt

,city_id,city_name,state_id,state_name,population,health_system_region,last_updated_number_beds,author_number_beds,number_beds,last_updated_number_ventilators,author_number_ventilators,number_ventilators,confirmed_cases,active_cases,deaths,notification_rate,state_notification_rate,last_updated_subnotification,subnotification_place_type,subnotification_rate,subnotification_rank,subnotification_classification,last_updated_rt,rt_10days_ago_low,rt_10days_ago_high,rt_10days_ago_most_likely,rt_17days_ago_low,rt_17days_ago_high,rt_17days_ago_most_likely,rt_classification,rt_10days_ago_avg,rt_17days_ago_avg,rt_ratio_week_avg,rt_growth,last_updated_inloco,inloco_today_7days_avg,inloco_last_week_7days_avg,inloco_ratio_week_avgs,inloco_growth,dday_beds_best,dday_beds_worst,dday_classification,dday_beds_best_months,overall_alert,data_last_refreshed
0,1505536,Parauapebas,PA,Pará,208273,Serra dos Carajás,2020-04-01,DataSUS,182,2020-04-01,DataSUS,46.0,4325.0,3934.0,88.0,0.819842,0.294076,2020-06-10 12:53:24,city,0.180158,19.0,bom,2020-06-10 13:35:27,1.33,1.50,1.42,1.86,2.07,1.98,ruim,1.584286,2.084286,0.760110,melhorando,2020-06-10 12:54:43,0.386839,0.423252,0.913969,estabilizando,1.0,1.0,ruim,1.0,alto,2020-06-10 13:36:23
1,1504422,Marituba,PA,Pará,131521,Metropolitana,2020-04-01,DataSUS,106,2020-04-01,DataSUS,20.0,510.0,738.0,50.0,0.206007,0.294076,2020-06-10 12:53:24,city,0.793993,111.0,ruim,2020-06-10 13:35:27,1.41,1.66,1.56,1.84,2.17,2.02,ruim,1.742857,2.245714,0.776081,melhorando,2020-06-10 12:54:43,0.379520,0.403175,0.941329,estabilizando,1.0,1.0,ruim,1.0,alto,2020-06-10 13:36:23
2,1501808,Breves,PA,Pará,102701,Marajó Ocidental,2020-04-01,DataSUS,88,2020-04-01,DataSUS,26.0,649.0,912.0,63.0,0.194036,0.294076,2020-06-10 12:53:24,city,0.805964,117.0,ruim,2020-06-10 13:35:27,0.94,1.14,1.06,1.21,1.41,1.32,insatisfatório,1.155714,1.484286,0.778633,melhorando,2020-06-10 12:54:43,0.440908,0.480991,0.916665,estabilizando,1.0,1.0,ruim,1.0,alto,2020-06-10 13:36:23
3,2306405,Itapipoca,CE,Ceará,129358,Itapipoca,2020-04-01,DataSUS,85,2020-04-01,DataSUS,20.0,1236.0,1450.0,65.0,0.369001,0.307393,2020-06-10 12:53:24,city,0.630999,84.0,insatisfatório,2020-06-10 13:35:27,1.27,1.48,1.38,1.54,1.78,1.67,ruim,1.457143,1.824286,0.798747,melhorando,2020-06-10 12:54:43,0.470121,0.509744,0.922268,estabilizando,1.0,1.0,ruim,1.0,alto,2020-06-10 13:36:23
4,1501402,Belém,PA,Pará,1492745,Metropolitana,2020-04-01,DataSUS,2957,2020-04-01,DataSUS,732.0,14880.0,25697.0,1644.0,0.180835,0.294076,2020-06-10 12:53:24,city,0.819165,118.0,ruim,2020-06-10 13:35:27,1.02,1.10,1.07,1.49,1.58,1.54,insatisfatório,1.292857,1.617143,0.799470,melhorando,2020-06-10 12:54:43,0.409748,0.446016,0.918685,estabilizando,1.0,1.0,ruim,1.0,alto,2020-06-10 13:36:23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,3518701,Guarujá,SP,São Paulo,320459,Baixada Santista,2020-04-01,DataSUS,336,2020-04-01,DataSUS,144.0,1917.0,2907.0,81.0,0.468810,0.307050,2020-06-10 12:53:24,city,0.531190,62.0,insatisfatório,2020-06-10 13:35:27,1.54,1.80,1.69,1.22,1.54,1.40,ruim,1.538571,1.381429,1.113754,piorando,2020-06-10 12:54:43,0.428972,0.438579,0.978094,estabilizando,1.0,1.0,ruim,1.0,alto,2020-06-10 13:36:23
123,2504009,Campina Grande,PB,Paraíba,409731,III Núcleo Regional de Saúde,2020-04-01,DataSUS,1087,2020-04-01,DataSUS,170.0,2967.0,2031.0,45.0,1.000000,0.818477,2020-06-10 12:53:24,city,0.000000,6.0,bom,2020-06-10 13:35:27,1.61,1.91,1.78,1.59,2.01,1.82,ruim,1.828571,1.632857,1.119860,piorando,2020-06-10 12:54:43,0.415434,0.470505,0.882953,piorando,27.0,21.0,ruim,1.0,alto,2020-06-10 13:36:23
124,3548807,São Caetano do Sul,SP,São Paulo,161127,Grande São Paulo,2020-04-01,DataSUS,708,2020-04-01,DataSUS,103.0,885.0,1250.0,51.0,0.362295,0.307050,2020-06-10 12:53:24,city,0.637705,102.0,insatisfatório,2020-06-10 13:35:27,1.31,1.58,1.46,1.12,1.43,1.28,ruim,1.374286,1.221429,1.125146

Quantidade de municípios por estado presente na lista

In [10]:
x = list(set(dfnfilt['state_id']))
dict(zip(x,[sum(dfnfilt['state_id']==i) for i in x]))

{'DF': 1,
 'ES': 4,
 'AC': 1,
 'SP': 33,
 'AL': 1,
 'PA': 17,
 'PR': 1,
 'MA': 4,
 'CE': 8,
 'GO': 1,
 'RN': 2,
 'AM': 7,
 'SE': 1,
 'RS': 2,
 'RO': 1,
 'RR': 1,
 'BA': 2,
 'PI': 1,
 'RJ': 19,
 'PE': 13,
 'AP': 2,
 'MG': 2,
 'PB': 3}

In [11]:
#Calculo de mortes/100k Hab.
def deaths_per_100khab(df, col_deaths="deaths",col_pop='population'):
    return 100e3 * df[col_deaths]/df[col_pop]

dfnfilt['deaths_100khab'] = deaths_per_100khab(dfnfilt)
dfnfilt['cases_100khab'] = deaths_per_100khab(dfnfilt, col_deaths="confirmed_cases")

Salvando os dados como desmontrando anteriormente mais as colunas de mortes e casos por 100 mil habitantes em .csv

In [12]:
dfnfilt[['city_name','deaths','deaths_100khab',"inloco_today_7days_avg","inloco_last_week_7days_avg", "inloco_ratio_week_avgs",'rt_17days_ago_avg','rt_10days_ago_avg','rt_ratio_week_avg']].to_csv("../data/output/dfnfilt.csv", index=False)

DataFrame com os dados ordenados. 
Alterando o nome da coluna, altera a ordenação dos dados, as 10 primeiras linhas serão
as 10 menores observações, as 10 ultimas, serão as 10 maiores
e.g. Municipios com o maior Rt, municípios com o menor Rt ...

In [13]:
dfexib = dfnfilt.sort_values('rt_10days_ago_avg')#Try me

### Visualização da relação do Rt com outras variáveis


In [14]:
# Gráfico de Dispersão entre variáveis
## Usar para análise exploratória
x = 'deaths_100khab' #Try me
y = 'rt_ratio_week_avg' #Try me
z = dfexib.sort_values(x)

fig = go.Figure()
fig.add_trace(
    go.Scatter(x=list(z[x]), y=list(z[y]),
               mode='markers',text=z['city_name']))


fig.layout.xaxis.update({'title': "Mortes por 100 mil habitantes"})
fig.layout.yaxis.update({'title': 'Razão de crescimento do ritmo de contágio médio'})
fig.update_layout(title= ' $$Gráfico\\ do\\ ΔR_t \\ versus\\ Mortes \\ por \\ 100\\ mil\\ hab$$')
fig.show()

Lista dos municípios que apresentam Rt < 1:

In [15]:
dfrtl = dfnna.query("rt_10days_ago_avg < 1")\
            .reset_index(drop=True)\
            .sort_values(['rt_10days_ago_avg'])

dfrtl

,city_id,city_name,state_id,state_name,population,health_system_region,last_updated_number_beds,author_number_beds,number_beds,last_updated_number_ventilators,author_number_ventilators,number_ventilators,confirmed_cases,active_cases,deaths,notification_rate,state_notification_rate,last_updated_subnotification,subnotification_place_type,subnotification_rate,subnotification_rank,subnotification_classification,last_updated_rt,rt_10days_ago_low,rt_10days_ago_high,rt_10days_ago_most_likely,rt_17days_ago_low,rt_17days_ago_high,rt_17days_ago_most_likely,rt_classification,rt_10days_ago_avg,rt_17days_ago_avg,rt_ratio_week_avg,rt_growth,last_updated_inloco,inloco_today_7days_avg,inloco_last_week_7days_avg,inloco_ratio_week_avgs,inloco_growth,dday_beds_best,dday_beds_worst,dday_classification,dday_beds_best_months,overall_alert,data_last_refreshed
1,4213500,Porto Belo,SC,Santa Catarina,21388,Balneário Camboriú,2020-04-01,DataSUS,0,2020-04-01,DataSUS,0.0,28.0,35.0,1.0,0.508571,1.000000,2020-06-10 12:53:24,city,0.491429,34.0,bom,2020-06-10 13:35:27,0.57,1.50,1.04,0.37,1.31,0.84,insatisfatório,0.907143,0.852857,1.063652,estabilizando,2020-06-10 12:54:43,0.379673,0.401990,0.944485,estabilizando,1.0,1.0,ruim,1.0,alto,2020-06-10 13:36:23
0,4314902,Porto Alegre,RS,Rio Grande do Sul,1483771,CRS 01 Porto Alegre,2020-04-01,DataSUS,4912,2020-04-01,DataSUS,1374.0,1153.0,1142.0,46.0,0.481564,0.838576,2020-06-10 12:53:24,city,0.518436,45.0,insatisfatório,2020-06-10 13:35:27,0.88,1.23,1.07,0.82,1.18,1.02,insatisfatório,0.990000,1.054286,0.939024,estabilizando,2020-06-10 12:54:43,0.420560,0.433076,0.971099,estabilizando,91.0,91.0,bom,3.0,medio,2020-06-10 13:36:23


Em apenas {{dfrtl.shape[0]}} município(s) o Rt está abaixo de 0

## Carregando dados com a série histórica da doença

In [16]:
dfh = pd.read_csv('http://datasource.coronacidades.org:7000/br/cities/cases/full')

In [17]:
dfh

,city,city_id,last_updated,epidemiological_week,estimated_population_2019,is_last,is_repeated,confirmed_cases,last_available_confirmed_per_100k_inhabitants,last_available_date,last_available_death_rate,deaths,daily_cases,new_deaths,order_for_place,place_type,state,infectious_period_cases,state_notification_rate,notification_rate,active_cases,data_last_refreshed
0,Alta Floresta D'Oeste,1100015,2020-05-02,18,22945.0,False,False,1,4.35825,2020-05-02,0.0000,0,1.0,0,1,city,RO,1.0,0.716890,0.716890,1.0,2020-06-10 15:16:15
1,Alta Floresta D'Oeste,1100015,2020-05-03,19,22945.0,False,False,1,4.35825,2020-05-03,0.0000,0,0.0,0,2,city,RO,1.0,0.685501,0.685501,1.0,2020-06-10 15:16:15
2,Alta Floresta D'Oeste,1100015,2020-05-04,19,22945.0,False,False,1,4.35825,2020-05-04,0.0000,0,0.0,0,3,city,RO,1.0,0.647368,0.647368,2.0,2020-06-10 15:16:15
3,Alta Floresta D'Oeste,1100015,2020-05-05,19,22945.0,False,False,1,4.35825,2020-05-05,0.0000,0,0.0,0,4,city,RO,1.0,0.614688,0.614688,2.0,2020-06-10 15:16:15
4,Alta Floresta D'Oeste,1100015,2020-05-06,19,22945.0,False,False,1,4.35825,2020-05-06,0.0000,0,0.0,0,5,city,RO,1.0,0.608108,0.608108,2.0,2020-06-10 15:16:15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181495,Brasília,5300108,2020-06-05,23,3015268.0,False,False,12864,426.62874,2020-06-05,0.0145,186,1166.0,5,91,city,DF,7527.0,1.000000,1.000000,7527.0,2020-06-10 15:16:15
181496,Brasília,5300108,2020-06-06,23,3015268.0,False,False,14330,475.24797,2020-06-06,0.0134,192,1466.0,6,92,city,DF,8725.0,1.000000,1.000000,8725.0,2020-06-10 15:16:15
181497,Brasília,5300108,2020-06-07,24,3015268.0,False,False,15015,497.96569,2020-06-07,0.0131,196,685.0,4,93,city,DF,9113.0,1.000000,1.000000,9113.0,2020-06-10 15:16:15
181498,Brasília,5300108,2020-06-08,24,3015268.0,False,False,15015,497.96569,2020-06-08,0.0131,196,0.0,0,94,city,DF,8818.0,1.000000,1.000000,8818.0,2020-06-10 15:16:15


In [18]:
print("Atualizado pela última vez em: " + str(dfh['last_updated'].max()))

Atualizado pela última vez em: 2020-06-09


<h4>Montando o heatmap de mortes normalizadas por município</h4>
Cálculo da média móvel das novas mortes nos últimos 5 dias

In [19]:
def get_rolling_amount(grp):
    return grp.rolling(5, min_periods=1, on='last_updated')['new_deaths'].mean()


df_cities_deaths = dfh[~dfh['deaths'].isnull()][['state','city','city_id', 'last_updated', 'deaths','new_deaths']]\
                                                .groupby(['state','city','city_id', 'last_updated'])[['deaths','new_deaths']]\
                                                .sum()\
                                                .reset_index()

df_cities_deaths['rolling_deaths_new'] = df_cities_deaths.groupby('city', 
                                                    as_index=False, 
                                                    group_keys=False)\
                                           .apply(get_rolling_amount)

In [20]:
#Nome das cidades por ordem (Pode variar de acordo com o que quisermos listar)
##(e.g. Variação do Rt ou valor absoluto do Rt)
cities_names_all = list(dfnfilt['city_name'])

cities_names = list(dfnfilt.sort_values('rt_10days_ago_avg', ascending = False)\
                                             ['city_name'].iloc[:10])

#ID da cidades por ordem (Pode variar de acordo com o que quisermos listar)
##(e.g. Variação do Rt ou valor absoluto do Rt)
cities_id_all = list(dfnfilt['city_id'])

cities_id = list(dfnfilt.sort_values('rt_10days_ago_avg', ascending = False)\
                                             ['city_id'].iloc[:10])

#Selecionar apesar os municípios listados em city_id
t = df_cities_deaths.query(f'city_id =={cities_id}')


Primeira morte e primeiro caso em cada cidade:

In [21]:
first_death = [dfh.query(f'city_id =={j}')\
    .query(f'deaths !=0')['last_updated'].min() for j in cities_id_all 
]

first_case = [dfh.query(f'city_id =={j}')\
    .query(f'confirmed_cases != 0')['last_updated'].min() for j in cities_id_all 
]

In [22]:
#Titulos dos gráficos
titles = [
    'Distribuição de mortes nas 10 cidades com o menor ritmo de contágio',
    'Distribuição de mortes nas 10 cidades com o maior ritmo de contágio',
    'Distribuição de mortes nas 10 cidades com o maior diminuição ritmo de contágio',
    'Distribuição de mortes nas 10 cidades com o maior aumento ritmo de contágio'
]

In [23]:
v = 0 #Contagem de quantas imagens foram salvas (para uso posterior)

In [24]:
place_type='city'
col_time= 'last_updated'
#min_deaths=10
title="Mapa de calor: Média móvel normalizada de novas mortes nos últimos 5 dias"
colors='temps'

In [25]:
#Formatar o Dataset para a plotagem
df_heatmap = t.reset_index()\
                              .pivot(index='city', 
                                      columns='last_updated', 
                                      values='rolling_deaths_new')\
                               .fillna(0)\
                               .apply(lambda x: x/x.max(), axis=1)\
                               .dropna(how='all')

# remove dias com todos os valores iguais a 0
df_heatmap = df_heatmap.loc[:, (df_heatmap != 0).any(axis=0)]
print(len(df_heatmap))

city_total_deaths = [t[t['city'] ==x]['deaths'].max() for x in df_heatmap.loc[cities_names].index]



10


### Visualização da distribuições das mortes nos municípios selecionados

In [26]:
df_heatmap.loc[cities_names]

last_updated,2020-04-02,2020-04-07,2020-04-13,2020-04-14,2020-04-15,2020-04-16,2020-04-17,2020-04-18,2020-04-19,2020-04-20,2020-04-21,2020-04-22,2020-04-23,2020-04-24,2020-04-25,2020-04-26,2020-04-27,2020-04-28,2020-04-29,2020-04-30,2020-05-01,2020-05-02,2020-05-03,2020-05-04,2020-05-05,2020-05-06,2020-05-07,2020-05-08,2020-05-09,2020-05-10,2020-05-11,2020-05-12,2020-05-13,2020-05-14,2020-05-15,2020-05-16,2020-05-17,2020-05-18,2020-05-19,2020-05-20,2020-05-21,2020-05-22,2020-05-23,2020-05-24,2020-05-25,2020-05-26,2020-05-27,2020-05-28,2020-05-29,2020-05-30,2020-05-31,2020-06-01,2020-06-02,2020-06-03,2020-06-04,2020-06-05,2020-06-06,2020-06-07,2020-06-08,2020-06-09
city,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Barcarena,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.021277,0.021277,0.021277,0.021277,0.021277,0.000000,0.000000,0.000000,0.042553,0.042553,0.042553,0.042553,0.042553,0.000000,0.042553,0.042553,0.042553,0.085106,0.106383,0.063830,0.063830,0.063830,0.021277,0.000000,0.063830,0.063830,0.787234,0.808511,0.978723,1.000000,1.000000,0.297872,0.319149,0.148936,0.063830,0.042553,0.063830,0.021277,0.021277,0.063830,0.063830,0.021277,0.021277,0.127660,0.085106,0.106383,0.191489
Paragominas,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.072464,0.108696,0.086957,0.130435,0.130435,0.086957,0.043478,0.043478,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.260870,0.478261,0.478261,0.695652,0.695652,0.521739,0.956522,0.956522,0.782609,1.000000,0.913043,0.260870,0.304348,0.652174,0.608696,0.608696,0.608696,0.826087,0.434783,0.347826,0.347826
Curuçá,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.176471,0.176471,0.176471,0.176471,0.176471,0.000000,0.000000,0.411765,0.411765,0.411765,0.411765,0.470588,0.058824,0.058824,0.058824,0.058824,0.705882,0.705882,0.882353,0.941176,1.000000,0.411765,0.411765,0.470588,0.411765,0.411765,0.352941,0.352941,0.117647,0.117647,0.058824,0.000000,0.000000,0.176471,0.176471,0.235294,0.235294
Abaetetuba,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.043478,0.043478,0.130435,0.260870,0.304348,0.347826,0.347826,0.260870,0.173913,0.130435,0.130435,0.130435,0.260870,0.304348,0.695652,0.913043,0.913043,0.869565,1.000000,0.869565,0.652174,0.782609,0.739130,0.521739,0.260870,0.478261,0.478261,0.434783,0.434783,0.565217,0.260870,0.260870,0.260870
Tucuruí,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.045455,0.045455,0.136364,0.136364,0.136364,0.090909,0.227273,0.136364,0.136364,0.136364,0.136364,0.045455,0.045455,0.045455,0.045455,0.500000,0.681818,0.681818,1.000000,1.000000,0.727273,0.500000,0.500000,0.454545,0.500000,0.318182,0.590909,0.590909,0.318182,0.272727,0.272727,0.000000,0.136364,0.136364,0.136364,0.318182,0.363636,0.454545,0.545455
São Vicente,0.023256,-0.023256,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.023256,0.000000,0.023256,0.023256,0.023256,0.000000,0.023256,0.000000,0.000000,0.023256,0.023256,0.023256,0.069767,0.069767,0.046512,0.046512,0.046512,0.023256,0.093023,0.116279,0.139535,0.139535,0.116279,0.046512,0.069767,0.093023,0.093023,0.116279,0.116279,0.209302,0.348837,0.348837,0.372093,0.441860,0.325581,0.348837,0.488372,0.418605,0.813953,1.000000,0.790698,0.697674,0.720930,0.255814,0.139535,0.186047,0.232558,0.255814,0.255814,

In [27]:
# Plotar o mapa de calor com total de mortes por município
fig = heatmap(df_heatmap.loc[cities_names].iloc[::-1], city_total_deaths[::-1], title, colors)
fig.show()

In [28]:
import plotly.io as pio

In [29]:
#Salvar imagem (discomente as linhas abaixo se quiser salvar a figura)
#pio.write_image(fig,f'../data/figures/hm0{v}.jpeg',format='jpeg',width=1920, height=1080)
#v+=1

In [30]:
dfnfilt['first_case'] = first_case
dfnfilt['first_death'] = first_death

<h4>DataFrame com os dados ordenados</h4>
Alterando o nome da coluna, altera a ordenação dos dados, as 10 primeiras linhas serão
as 10 menores observações, as 10 ultimas, serão as 10 maiores
e.g. Municipios com o maior Rt, municípios com o menor Rt ...

In [31]:
dfexib = dfnfilt.sort_values('rt_10days_ago_avg', ascending = False) #Alterar essa linha

In [32]:
h = dfexib[['city_name','state_id','first_death','deaths','deaths_100khab',
 'first_case','confirmed_cases', 'cases_100khab', 'subnotification_rate',
        'inloco_last_week_7days_avg','inloco_today_7days_avg',
        'rt_17days_ago_avg','rt_10days_ago_avg']]
h.to_csv("../data/output/dados_g1.csv", index=False)

Exibição dos dados mostrando algumas variáveis relevantes.
No exemplo, municípios com os 10 menores Rt

In [33]:
h.iloc[:10]

,city_name,state_id,first_death,deaths,deaths_100khab,first_case,confirmed_cases,cases_100khab,subnotification_rate,inloco_last_week_7days_avg,inloco_today_7days_avg,rt_17days_ago_avg,rt_10days_ago_avg
105,Barcarena,PA,2020-04-24,72.0,57.747834,2020-04-01,1006.0,806.865576,0.728585,0.402272,0.374460,2.220000,2.260000
126,Paragominas,PA,2020-04-26,64.0,56.564585,2020-04-24,955.0,844.049671,0.726246,0.437809,0.405599,1.402857,2.244286
48,Curuçá,PA,2020-05-04,40.0,99.835272,2020-04-22,501.0,1250.436779,0.775231,0.450519,0.404342,2.180000,2.000000
125,Abaetetuba,PA,2020-05-08,68.0,43.120395,2020-03-31,1735.0,1100.204188,0.476591,0.477182,0.444223,1.648571,1.970000
12,Tucuruí,PA,2020-05-02,61.0,53.669309,2020-03-31,722.0,635.233461,0.765257,0.462946,0.428964,2.311429,1.967143
121,São Vicente,SP,2020-04-02,99.0,27.064117,2020-03-31,1377.0,376.437269,0.718989,0.423344,0.409233,1.667143,1.855714
119,Benevides,PA,2020-04-24,43.0,68.540096,2020-04-03,1083.0,1726.254045,0.538777,0.407562,0.375367,1.702857,1.835714
123,Campina Grande,PB,2020-04-16,45.0,10.982816,2020-03-27,2967.0,724.133639,0.000000,0.470505,0.415434,1.632857,1.828571
113,Magé,RJ,2020-04-13,98.0,39.988412,2020-04-02,961.0,392.131260,0.827039,0.436889,0.423475,1.731429,1.795714
1,Marituba,PA,2020-04-16,50.0,38.016743,2020-03-31,510.0,387.770774,0.793993,0.403175,0.379520,2.245714,1.742857


In [34]:
print("Maior Rt: %f" % dfnfilt['rt_10days_ago_avg'].sort_values(ascending = False)[:10].max())
print("Menor Rt: %f" % dfnfilt['rt_10days_ago_avg'].sort_values(ascending = False)[:10].min())

Maior Rt: 2.260000
Menor Rt: 1.742857
